In [3]:
import pandas as pd
import json

prompts, names, runs = {}, {}, []

with open("main.json.log", "r") as file:
    for line in file:
        item = json.loads(line)

        if item["type"] == "entity":
            names[item["key"]] = item["name"]
        elif item["type"] == "prompt":
            prompts[item["prompt_id"]] = item
        elif item["type"] == "run" and "result" in item:
            result = item["result"]
            item["text"] = (result["choices"][0] if "choices" in result else result)["message"]["content"]
            item["len"] = len(item["text"]) + len(prompts[item["prompt_id"]]["prompt"])
            runs.append(item)

df = pd.DataFrame(runs)

In [ ]:
df["metric"] = df["len"] / df["elapsed_time"]

table = df.groupby(["runner", "model"])["metric"].median().reset_index() \
    .pivot(index="model", columns="runner", values="metric")

table.index.name = None
table.columns.name = None
name_order = list(names.keys())

def sort_key(value):
    return name_order.index(value) if value in name_order else float('inf')

table = table.loc[sorted(table.index, key=sort_key)]
table = table[sorted(table.columns, key=sort_key)]
table.index = table.index.map(lambda x: names.get(x, x))
table.columns = table.columns.map(lambda x: names.get(x, x))
table = table.applymap(lambda x: int(x) if pd.notna(x) else "")

table